## 기본세팅

In [ ]:
!apt-get update 
!apt-get install g++ openjdk-8-jdk python-dev python3-dev 
!pip3 install JPype1-py3 
!pip3 install konlpy
!pip3 install WordCloud

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
from konlpy.utils import pprint
from konlpy.tag import Kkma
from konlpy.tag import Okt
from konlpy.tag import Twitter
from collections import Counter
import os
import sys
import urllib.request
import ssl
import json
import pandas as pd
from wordcloud import WordCloud
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import re
plt.rc('font',family='NanumBarunGothic')
#cell magic 명령어 : 그래프를 출력영역에 표시
%matplotlib inline 

## 파일 불러오기

+ 인스타 크롤링 데이터

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
apple_insta = pd.read_csv('/content/drive/MyDrive/Final/Apple(insta).csv', index_col = 0)
apple_insta.head()

In [ ]:
mi_insta = pd.read_csv('/content/drive/MyDrive/Final/Mi(insta).csv', index_col = 0)
mi_insta.head()

In [ ]:
samsung_insta = pd.read_csv('/content/drive/MyDrive/Final/Samsung(insta).csv', index_col = 0)
samsung_insta.head()

## 함수

In [ ]:
##### 한글을 제외한 문자 제거
def clean_str(s):
  hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')    #한글만 남겨놓고 나머지 삭제
  s = hangul.sub(r'',s)

  cp = re.compile("["
        u"\U00010000-\U0010FFFF"  #BMP characters 이외 이모티콘삭제
                           "]+", flags=re.UNICODE)
  s=cp.sub(r'',s)

  s = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', s) #특수기호 삭제
 
  return s.strip()

In [ ]:
##### 명사 갯수 추출  
def get_tags(text, ntags=50):   #클로링한 하나의 문장에서 명사를 축출해서 명사의 갯수를 딕셔너리로 리턴한다.
    spliter = Okt()
    nouns = spliter.nouns(text)
    count = Counter(nouns)
    return_list = []
    for n, c in count.most_common(ntags):  #count로 단어의 갯수를 파악한걸 정렬해서 ntags갯수만큼 리스트로 가져온다.
        temp = {'tag': n, 'count': c}
        return_list.append(temp)
    #return_list
    return dict(count.most_common(ntags))    #  [('커피',1200),('카페',800)]   ->  {'커피':1200 , '카페':800}

In [ ]:
##### 워드클라우드 생성
def Showwordcloud(data,savename,maskname=''):
    #워드 클라우드 마스크기능 사용하기
    #colormap 확인 가능 블로그 : https://pythonkim.tistory.com/82
    from PIL import Image
    if maskname=='':
        wc = WordCloud(font_path='/content/drive/MyDrive/Final/NanumBarunGothicBold.ttf',background_color='white',max_font_size=60
                , colormap='autumn')
    else:
        maskimg = np.array(Image.open(maskname))
        wc = WordCloud(font_path='/content/drive/MyDrive/Final/NanumBarunGothicBold.ttf',background_color='white',max_font_size=60
                , mask=maskimg , colormap='autumn')
    
    wc.generate_from_frequencies(data)
    plt.figure(figsize=(8,5))
    plt.imshow(wc)
    plt.tight_layout(pad=0)
    plt.axis('off')
    plt.show()
    wc.to_file(savename+'.png')


## 워드클라우드 생성

In [ ]:
##### 명사 추출을 위해 데이터를 리스트에 문자열로 추가

test = []
for i in apple_insta["context"]:
    teststr = ''
    teststr = teststr + i
    test.append(teststr)
print(test)

In [ ]:
data = apple_insta

In [ ]:
print(data.columns)
strdata = ''
replydata = list()
for temp in data["context"]:
    strdata = strdata + temp
    replydata.append(temp)

strdata = strdata.replace('\n','')
strdata = strdata.strip()
strdata = strdata.replace('0xed','')
strdata = strdata.replace('#','')

In [ ]:
strdata = clean_str(strdata) #이모티콘 특수기호 삭제
resultdata = get_tags(strdata,100)
Showwordcloud(resultdata,'삼성')

In [ ]:
# 삼성 키워드 제거
del resultdata['지원']
del resultdata['힐앤힐']
del resultdata['원주시']
del resultdata['원주']
del resultdata['이벤트']
del resultdata['아저씨']
del resultdata['수리']
del resultdata['버즈']
del resultdata['갤럭시탭']
del resultdata['갤탭']
del resultdata['샵']
del resultdata['시계']
del resultdata['맞팔']
del resultdata['오늘']

In [ ]:
# 애플 키워드 제거

del resultdata["힐앤힐"]
del resultdata["원주"]
del resultdata["맞팔"]
del resultdata["아저씨"]
del resultdata["다이어리"]
del resultdata["원주시"]
del resultdata["소통"]
del resultdata["그램"]
del resultdata["피드"]
del resultdata["스트랩"]
del resultdata["파우치"]
del resultdata["이벤트"]
del resultdata["아이패드"]
del resultdata["워치"]
del resultdata["에어팟"]
del resultdata["맥북"]
del resultdata["아이맥"]
del resultdata["선팔"]
del resultdata["프로"]

In [ ]:
resultdata
sorted(resultdata.items(), reverse=True, key=lambda item: item[1])

In [ ]:
##### apple get tags 저장
apple_result = resultdata

In [ ]:
##### samsung get tags 저장
samsung_result = resultdata